In [6]:
# Install scikit-survival package
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-survival"])

0

In [7]:
import pandas as pd
import numpy as np

%pip install scikit-learn==1.3.2 scikit-survival==0.22.2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from lifelines import CoxPHFitter, WeibullAFTFitter
from lifelines.utils import concordance_index
from sksurv.util import Surv
from sksurv.metrics import (
    concordance_index_ipcw,
    integrated_brier_score,
    brier_score,
    cumulative_dynamic_auc
)

# Load datasets
df_zero    = pd.read_csv("C:\\Users\\04ama\\OneDrive\\pension survival analysis\\notebooks\\ipcw_and_other_censoring\\data\\censoring_methods\\data_zero.csv")
df_discard = pd.read_csv("C:\\Users\\04ama\\OneDrive\\pension survival analysis\\notebooks\\ipcw_and_other_censoring\\data\\censoring_methods\\data_discard.csv")
df_ipcw    = pd.read_csv("C:\\Users\\04ama\\OneDrive\\pension survival analysis\\notebooks\\ipcw_and_other_censoring\\data\\censoring_methods\\data_ipcw.csv")

datasets = {"zero": df_zero, "discard": df_discard, "ipcw": df_ipcw}

X_COLS = ["age_at_entry", "income_level", "health_score", "pension_contrib_rate"]
DUR = "time_to_event"
EVT = "event_observed"
T_STAR = 15.0
# Adjust TIMES to be within the valid follow-up range [0.04; 25.0[
TIMES = np.linspace(1, 24, 6)  # Changed from (5, 30, 6) to (1, 24, 6)


Note: you may need to restart the kernel to use updated packages.


In [8]:
from lifelines import KaplanMeierFitter
from sksurv.metrics import concordance_index_ipcw, integrated_brier_score, cumulative_dynamic_auc

def make_surv(df):
    return Surv.from_arrays(event=df[EVT].astype(bool), time=df[DUR])

def evaluate_survival_model(model_name, model, train_df, test_df, weights=None):
    y_tr = make_surv(train_df)
    y_te = make_surv(test_df)
    S_pred = model.predict_survival_function(test_df[X_COLS], times=TIMES).T.values
    risk_scores = 1 - S_pred[:, -1]  # event risk at last time
    c_uno = concordance_index_ipcw(y_tr, y_te, -risk_scores, tau=TIMES[-1])[0]
    ibs = integrated_brier_score(y_tr, y_te, S_pred, TIMES)
    auc_times, aucs = cumulative_dynamic_auc(y_tr, y_te, risk_scores, TIMES)
    
    # Handle case where aucs might be scalar or array
    if np.isscalar(aucs):
        auc_15 = float(aucs)
    else:
        # Find closest time to T_STAR
        closest_idx = np.argmin(np.abs(auc_times - T_STAR))
        auc_15 = float(aucs[closest_idx])
    
    return {"Model": model_name, "C_index": c_uno, "IBS": ibs, "AUC@15": auc_15}

def evaluate_classifier(model_name, model, X_train, y_train, X_test, y_test, sample_weight=None):
    """Evaluate binary classifier"""
    # Fit model
    if sample_weight is not None and hasattr(model, 'fit') and 'sample_weight' in model.fit.__code__.co_varnames:
        model.fit(X_train, y_train, sample_weight=sample_weight)
    else:
        model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_pred_proba)
    elif hasattr(model, 'decision_function'):
        y_pred_scores = model.decision_function(X_test)
        auc = roc_auc_score(y_test, y_pred_scores)
    else:
        auc = np.nan
    
    f1 = f1_score(y_test, y_pred)
    
    return {"Model": model_name, "Accuracy": accuracy, "AUC": auc, "F1": f1}


In [9]:
# Add missing import for brier_score_loss
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, brier_score_loss

all_results = []
baseline_results = {}  # Store baseline predictions for NRI

for method, df in datasets.items():
    print(f"\n=== METHOD: {method.upper()} ===")
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[EVT])

    df_fit = train_df[[DUR, EVT] + X_COLS].copy()
    
    # ------------- SURVIVAL MODELS -------------
    if method == "ipcw" and "ipcw" in train_df.columns:
        df_fit["ipcw"] = train_df["ipcw"]
        w_col = "ipcw"
    else:
        w_col = None

    # Cox PH
    cph = CoxPHFitter()
    cph.fit(df_fit, duration_col=DUR, event_col=EVT,
            weights_col=w_col if w_col else None)
    
    # Survival model evaluation
    y_tr = make_surv(train_df)
    y_te = make_surv(test_df)
    S_pred = cph.predict_survival_function(test_df[X_COLS], times=TIMES).T.values
    risk_scores = 1 - S_pred[:, -1]
    c_uno = concordance_index_ipcw(y_tr, y_te, -risk_scores, tau=TIMES[-1])[0]
    ibs = integrated_brier_score(y_tr, y_te, S_pred, TIMES)
    auc_times, aucs = cumulative_dynamic_auc(y_tr, y_te, risk_scores, TIMES)
    
    if np.isscalar(aucs):
        auc_15 = float(aucs)
    else:
        closest_idx = np.argmin(np.abs(auc_times - T_STAR))
        auc_15 = float(aucs[closest_idx])
    
    res_cph = {"Model": "CoxPH", "Method": method, "C_index": c_uno, "IBS": ibs, "AUC@15": auc_15, "Model_Type": "Survival"}
    all_results.append(res_cph)

    # Weibull AFT
    aft = WeibullAFTFitter()
    aft.fit(df_fit[[DUR, EVT] + X_COLS], duration_col=DUR, event_col=EVT)
    
    # Weibull evaluation
    S_pred_aft = aft.predict_survival_function(test_df[X_COLS], times=TIMES).T.values
    risk_scores_aft = 1 - S_pred_aft[:, -1]
    c_uno_aft = concordance_index_ipcw(y_tr, y_te, -risk_scores_aft, tau=TIMES[-1])[0]
    ibs_aft = integrated_brier_score(y_tr, y_te, S_pred_aft, TIMES)
    auc_times_aft, aucs_aft = cumulative_dynamic_auc(y_tr, y_te, risk_scores_aft, TIMES)
    
    if np.isscalar(aucs_aft):
        auc_15_aft = float(aucs_aft)
    else:
        closest_idx_aft = np.argmin(np.abs(auc_times_aft - T_STAR))
        auc_15_aft = float(aucs_aft[closest_idx_aft])
    
    res_aft = {"Model": "WeibullAFT", "Method": method, "C_index": c_uno_aft, "IBS": ibs_aft, "AUC@15": auc_15_aft, "Model_Type": "Survival"}
    all_results.append(res_aft)

    # ------------- CLASSIFIERS -------------
    # Convert to binary outcome: event occurred by t*?
    y_train = ((train_df[DUR] <= T_STAR) & (train_df[EVT] == 1)).astype(int)
    y_test  = ((test_df[DUR] <= T_STAR) & (test_df[EVT] == 1)).astype(int)
    X_train, X_test = train_df[X_COLS], test_df[X_COLS]

    sw_train = None
    if method == "ipcw" and "ipcw" in train_df.columns:
        sw_train = train_df["ipcw"].copy()
        sw_train[y_train == 0] = 0  # only count events

    classifiers = {
        "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
        "Random Forest": RandomForestClassifier(n_estimators=400, random_state=42),
        "SVM (RBF)": make_pipeline(StandardScaler(), SVC(probability=True, random_state=42)),
        "KNN": make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=25))
    }

    for name, clf in classifiers.items():
        # Fit classifier
        if sw_train is not None and hasattr(clf, 'fit') and 'sample_weight' in clf.fit.__code__.co_varnames:
            clf.fit(X_train, y_train, sample_weight=sw_train)
        else:
            clf.fit(X_train, y_train)
        
        # Predictions
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Get probabilities
        if hasattr(clf, 'predict_proba'):
            y_pred_proba = clf.predict_proba(X_test)[:, 1]
        elif hasattr(clf, 'decision_function'):
            scores = clf.decision_function(X_test)
            y_pred_proba = 1 / (1 + np.exp(-scores))  # sigmoid
        else:
            y_pred_proba = y_pred.astype(float)
        
        # Calculate metrics
        auc = roc_auc_score(y_test, y_pred_proba)
        brier = brier_score_loss(y_test, y_pred_proba)
        c_index = auc  # C-index same as AUC for binary classification
        ibs_clf = brier  # IBS-like metric for classification
        
        # Calculate NRI (Net Reclassification Improvement)
        nri_total = np.nan
        if method != "zero" and f"{name}_zero_proba" in baseline_results:
            baseline_proba = baseline_results[f"{name}_zero_proba"]
            cutoff = 0.5
            
            # Old and new predictions
            y_pred_old = (baseline_proba >= cutoff).astype(int)
            y_pred_new = (y_pred_proba >= cutoff).astype(int)
            
            # NRI for events
            events_mask = y_test == 1
            if np.sum(events_mask) > 0:
                events_improved = np.sum((y_pred_new[events_mask] == 1) & (y_pred_old[events_mask] == 0))
                events_worsened = np.sum((y_pred_new[events_mask] == 0) & (y_pred_old[events_mask] == 1))
                nri_events = (events_improved - events_worsened) / np.sum(events_mask)
            else:
                nri_events = 0
            
            # NRI for non-events
            non_events_mask = y_test == 0
            if np.sum(non_events_mask) > 0:
                non_events_improved = np.sum((y_pred_new[non_events_mask] == 0) & (y_pred_old[non_events_mask] == 1))
                non_events_worsened = np.sum((y_pred_new[non_events_mask] == 1) & (y_pred_old[non_events_mask] == 0))
                nri_non_events = (non_events_improved - non_events_worsened) / np.sum(non_events_mask)
            else:
                nri_non_events = 0
            
            nri_total = nri_events + nri_non_events
        
        res_clf = {
            "Model": name, 
            "Method": method, 
            "Accuracy": accuracy, 
            "AUC": auc,
            "AUC@15": auc,
            "F1": f1,
            "C_index": c_index,
            "IBS": ibs_clf,
            "Brier_Score": brier,
            "NRI_total": nri_total,
            "Model_Type": "Classification"
        }
        all_results.append(res_clf)
        
        # Store baseline probabilities for NRI calculation
        if method == "zero":
            baseline_results[f"{name}_zero_proba"] = y_pred_proba

res = pd.DataFrame(all_results)
res.to_csv("results.csv", index=False)
print("\n✅ Enhanced results saved to results.csv")

# Display results by model type
print("\n=== SURVIVAL MODELS ===")
survival_res = res[res['Model_Type'] == 'Survival'][['Model', 'Method', 'C_index', 'IBS', 'AUC@15']]
display(survival_res)

print("\n=== CLASSIFICATION MODELS ===")
classification_res = res[res['Model_Type'] == 'Classification'][['Model', 'Method', 'C_index', 'IBS', 'AUC@15', 'NRI_total']]
display(classification_res)


=== METHOD: ZERO ===

=== METHOD: DISCARD ===

=== METHOD: DISCARD ===

=== METHOD: IPCW ===

=== METHOD: IPCW ===


c:\Users\04ama\anaconda3\Lib\site-packages\lifelines\fitters\coxph_fitter.py:1356: StatisticalWarning: It appears your weights are not integers, possibly propensity or sampling scores then?
It's important to know that the naive variance estimates of the coefficients are biased. Instead a) set `robust=True` in the call to `fit`, or b) use Monte Carlo to
estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"

  warnings.warn(



✅ Enhanced results saved to results.csv

=== SURVIVAL MODELS ===


,Model,Method,C_index,IBS,AUC@15
0,CoxPH,zero,0.374913,0.160489,0.677311
1,WeibullAFT,zero,0.374924,0.160390,0.677343
6,CoxPH,discard,0.374913,0.160489,0.677311
7,WeibullAFT,discard,0.374924,0.160390,0.677343
12,CoxPH,ipcw,0.383278,0.166091,0.665055
13,WeibullAFT,ipcw,0.383411,0.165075,0.664902



=== CLASSIFICATION MODELS ===


,Model,Method,C_index,IBS,AUC@15,NRI_total
2,Logistic Regression,zero,0.642143,0.223505,0.642143,NaN
3,Random Forest,zero,0.581362,0.242569,0.581362,NaN
4,SVM (RBF),zero,0.622832,0.229075,0.622832,NaN
5,KNN,zero,0.588189,0.234747,0.588189,NaN
8,Logistic Regression,discard,0.642143,0.223505,0.642143,0.000000
9,Random Forest,discard,0.581362,0.242569,0.581362,0.000000
10,SVM (RBF),discard,0.622832,0.229075,0.622832,0.000000
11,KNN,discard,0.588189,0.234747,0.588189,0.000000
14,Logistic Regression,ipcw,0.620438,0.228457,0.620438,0.071540
15,Random Forest,ipcw,0.553188,0.252480,0.553188,0.023841
